<a href="https://colab.research.google.com/github/jmarrietar/mineria-de-datos/blob/main/%5BProyecto%5D%20Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering 

TO DO: Dividir esta parte en dos: 

1. Clustering de Categorias (usando embeddings de los productos) y despues abajo clustering de los usuarios( Basado en lo que compran). 

# Clustering 

En la primera parte de este proyecto, se realizarán clusters o grupos de usuarios dependiendo de las compras realizadas. En la segunda sección se realizará un análisis de asociación con el cual se crearán reglas útiles para recomendar. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
import seaborn as sns
from os import path
from PIL import Image

import gdown
import matplotlib.pyplot as plt
% matplotlib inline

pd.set_option('display.max_rows', 100)

In [ ]:
!gdown https://drive.google.com/uc?id=1_AJd_FiQ6LSWSK4e__ccfLzhu_sWWNUS

Downloading...
From: https://drive.google.com/uc?id=1_AJd_FiQ6LSWSK4e__ccfLzhu_sWWNUS
To: /content/OnlineRetail.csv
45.6MB [00:00, 47.2MB/s]


In [ ]:
online_retail = pd.read_csv(
    "OnlineRetail.csv",
    sep=",",
    dtype={"CustomerID": "object"},
    encoding="unicode_escape",
)

In [ ]:
online_retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [ ]:
online_retail[["InvoiceNo","CustomerID", "StockCode", "Description"]]

,InvoiceNo,CustomerID,StockCode,Description
0,536365,17850,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,17850,71053,WHITE METAL LANTERN
2,536365,17850,84406B,CREAM CUPID HEARTS COAT HANGER
3,536365,17850,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,17850,84029E,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...
541904,581587,12680,22613,PACK OF 20 SPACEBOY NAPKINS
541905,581587,12680,22899,CHILDREN'S APRON DOLLY GIRL
541906,581587,12680,23254,CHILDRENS CUTLERY DOLLY GIRL
541907,581587,12680,23255,CHILDRENS CUTLERY CIRCUS PARADE


se utilizará la librería `gensim` para crear los embedding de los ítems con un modelo pre-entrenado 

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader
import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
!pip install spacy

In [ ]:
"""
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
# process a sentence using the model
doc = nlp("WHITE HANGING HEART T-LIGHT HOLDER")

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector
"""

array([ 1.9967848 , -1.4163905 , -0.44339842,  1.8873357 ,  0.07041259,
        2.7603605 ,  0.7227832 , -0.13723837,  1.9405673 ,  0.3631572 ,
        0.79684937,  0.07103961,  0.38307762, -0.5002725 , -0.90071404,
       -1.0240806 ,  0.5728866 ,  0.47602692,  0.532477  , -0.6861368 ,
        1.6059672 , -0.3004579 ,  0.15289983, -1.2232068 , -1.2062243 ,
       -0.99743205, -1.5930235 , -1.4195725 ,  1.5111408 , -0.37049612,
        0.6031875 , -1.1509011 ,  1.0975993 , -1.2982472 , -1.4535283 ,
       -2.1158073 ,  1.5569347 , -1.1844757 , -1.3063463 ,  0.91608506,
        2.9898553 ,  0.5089108 ,  0.01338463, -2.492564  , -0.61593103,
        0.39212388, -0.06552049,  0.32320324,  0.13534735,  0.728109  ,
        0.09116479, -0.7228257 , -1.989006  , -1.7347662 , -0.51813513,
        1.4201987 ,  0.7637191 ,  0.45090207,  0.0173033 ,  0.01277757,
       -0.21423952,  0.30137056,  0.88427407,  1.5848936 ,  2.1028557 ,
       -0.36031872,  0.85308075, -1.4747862 , -2.694639  ,  1.34

In [ ]:
online_retail[["InvoiceNo","CustomerID", "StockCode", "Description"]]

,InvoiceNo,CustomerID,StockCode,Description
0,536365,17850,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,17850,71053,WHITE METAL LANTERN
2,536365,17850,84406B,CREAM CUPID HEARTS COAT HANGER
3,536365,17850,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,17850,84029E,RED WOOLLY HOTTIE WHITE HEART.
...,...,...,...,...
541904,581587,12680,22613,PACK OF 20 SPACEBOY NAPKINS
541905,581587,12680,22899,CHILDREN'S APRON DOLLY GIRL
541906,581587,12680,23254,CHILDRENS CUTLERY DOLLY GIRL
541907,581587,12680,23255,CHILDRENS CUTLERY CIRCUS PARADE


In [ ]:
# ETA 2h aprox
# To Do: Incluir tqm
descriptor_embedding = []
for index, row in online_retail.iterrows():
    if row['Description'] is np.nan:
        doc = nlp('out of vocab')
    else:
        doc = nlp(row['Description'])
    descriptor_embedding.append(doc.vector)

In [ ]:
len(descriptor_embedding)

541909

In [ ]:
col_names = ["emb_" + str(i) for i in range(0, 96)]

In [ ]:
embeddings = pd.DataFrame(descriptor_embedding, columns = col_names)

In [ ]:
purchase_embeddings = pd.concat([online_retail[["CustomerID"]], embeddings], axis=1)

In [ ]:
# Create user embedding with mean of their Items purchase 
user_embeddings = purchase_embeddings.groupby("CustomerID").mean().reset_index()

In [ ]:
from sklearn import metrics, preprocessing
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [ ]:
# Normalize embeddings pre-clustering
X_norm = preprocessing.normalize(user_embeddings.iloc[:,1:100])

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
#plot_elbow(X_norm) # TO DO 

In [ ]:
# Create clusters
N_CLUSTERS = 3
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0).fit(X_norm)

# Assign cluster
user_embeddings['CLUSTER'] = kmeans.labels_

In [ ]:
# Metrics
cluster_metrics(user_embeddings.iloc[:,1:97], user_embeddings['CLUSTER'].values)

Silhoutte score: 0.114
Intra Cluster Silhoutte score:
cluster
0    0.726050
1    0.039407
2    0.161166
Name: silhouette_sample, dtype: float64
Davies Bouldin score: 2.514


In [ ]:
def plot_elbow(data):
    """Elbow curve"""
    model = KMeans()
    visualizer = KElbowVisualizer(model, k=(2, 60), timings=True)
    visualizer.fit(data)
    visualizer.show()


def cluster_metrics(X, labels):
    """Calculate Silhoutte & Davies Bouldin score"""

    X2 = X.copy()
    X2["cluster"] = labels
    X2["silhouette_sample"] = metrics.silhouette_samples(X, labels, metric="cosine")

    print(
        "Silhoutte score: {:.3f}".format(
            metrics.silhouette_score(X, labels, metric="cosine")
        )
    )
    print("Intra Cluster Silhoutte score:")
    print(X2.groupby("cluster")["silhouette_sample"].mean())
    print(
        "Davies Bouldin score: {:.3f}".format(metrics.davies_bouldin_score(X, labels))
    )


def plot_tsne(df, filename, hover_columns):
    """Plot TSNE"""

    print("Calculating 2D TSNE ...")
    X_tsne = TSNE(n_components=2).fit_transform(df.iloc[:, 1:97])
    df["tsne_0"] = X_tsne[:, 0]
    df["tsne_1"] = X_tsne[:, 1]
    fig = px.scatter(
        df, x="tsne_0", y="tsne_1", color="CLUSTER", hover_data=hover_columns
    )
    pio.write_html(fig, file="{}_2D.html".format(filename), auto_open=True)
    print("2D TSNE plot done!")

In [ ]:
import plotly.express as px
import plotly.io as pio

In [ ]:
# Plot Embeddings with cluster_id
hover_columns = ["CustomerID"]
plot_tsne(user_embeddings, filename="Cluster", hover_columns=hover_columns)

Calculating 2D TSNE ...
2D TSNE plot done!


In [ ]:
"""
TO DO: 
-----
    1. Elbow method hacer la grafica
    2. Analizar los clusters que se hicieron y darles un sentido. 

"""